In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# df_final = pd.read_csv("df_version_202505191015.csv")
df_final = pd.read_csv("general_info.csv")
df_filtered = (
    df_final
    .astype({"number_review":float})
    .loc[lambda df: df["number_review"]>100]
    .reset_index(drop=True)
    )
df_filtered

,company,website,rating,number_review
0,FlyDealFare.com,www.flydealfare.com,5.0,10378.0
1,Peru For Less,peruforless.com,5.0,1719.0
2,Rockymountaingetaways,www.rockymountaingetaways.com,5.0,1313.0
3,Monarch Air Group,monarchairgroup.com,5.0,959.0
4,Inca Expert Travel,incaexpert.com,5.0,783.0
...,...,...,...,...
1039,Bookingbuddy,www.bookingbuddy.com,1.1,145.0
1040,Tripping,tripping.com,1.1,144.0
1041,hotelvalue.com,hotelvalue.com,1.0,1165.0
1042,Colonial Properties Cabin & Resort Rentals,smokymtndreams.com,1.0,699.0


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re

# Leeres Dictionary mit Listen zur Datenspeicherung
df_prep = {
    "website": [],
    "5_star_percentage": [],
    "4_star_percentage": [],
    "3_star_percentage": [],
    "2_star_percentage": [],
    "1_star_percentage": [],    
}

# Basis-URL
url_base = "https://www.trustpilot.com/review/"

# Beispiel für df_filtered (falls noch nicht vorhanden)
# df_filtered = pd.DataFrame({"website": ["www.flydealfare.com", ...]})

for index, website in enumerate(df_filtered["website"].to_list()):
    print(index, website)
    
    url_complete = url_base + website
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

    # Wartezeit zur Umgehung von Blockierung
    # time.sleep(random.uniform(5, 15))
    
    try:
        html = requests.get(url_complete, headers=headers, timeout=10)
        if html.status_code != 200:
            print(f"Error loading {html.status_code} at {index, website}")
            break  # oder break, je nach gewünschtem Verhalten

        soup = BeautifulSoup(html.text, "lxml")

        field = soup.find("div", class_="paper_paper__EGeEb paper_outline__bqVmn card_card__yyGgu styles_reviewFilterCard__sn4Nz")
        
        if field:
            df_prep["website"].append(website)
            ratings = re.findall(r'\d+%|<1%', field.text)
            if len(ratings) == 5:
                df_prep["5_star_percentage"].append(ratings[0])
                df_prep["4_star_percentage"].append(ratings[1])
                df_prep["3_star_percentage"].append(ratings[2])
                df_prep["2_star_percentage"].append(ratings[3])
                df_prep["1_star_percentage"].append(ratings[4])
            else:
                print(f"⚠️ Unerwartete Anzahl an Bewertungen bei {website}: {ratings}")
                continue
        else:
            print(f"❌ Feld nicht gefunden bei {website}")
            break

    except Exception as e:
        print(f"🚨 Fehler bei {website}: {e}")
        continue

# DataFrame erzeugen
df_reviews = pd.DataFrame(df_prep)
display(df_reviews)
print(df_reviews.isna().sum())

0 www.flydealfare.com
Error loading 403 at (0, 'www.flydealfare.com')


,website,5_star_percentage,4_star_percentage,3_star_percentage,2_star_percentage,1_star_percentage


website              0.0
5_star_percentage    0.0
4_star_percentage    0.0
3_star_percentage    0.0
2_star_percentage    0.0
1_star_percentage    0.0
dtype: float64


In [4]:
df_reviews = pd.DataFrame(df_prep)
display(df_reviews)
print(df_reviews.isna().sum())

,website,5_star_percentage,4_star_percentage,3_star_percentage,2_star_percentage,1_star_percentage
0,www.flydealfare.com,95%,4%,<1%,<1%,<1%
1,peruforless.com,96%,3%,<1%,0%,0%
2,www.rockymountaingetaways.com,99%,<1%,<1%,0%,<1%
3,monarchairgroup.com,98%,2%,<1%,0%,<1%
4,incaexpert.com,98%,2%,0%,0%,0%
...,...,...,...,...,...,...
200,bigbearcoolcabins.com,63%,17%,10%,5%,5%
201,athotel.com,78%,10%,3%,2%,7%
202,furnishedquarters.com,62%,21%,8%,3%,6%
203,holidu.co.uk,65%,19%,6%,2%,8%


website              0
5_star_percentage    0
4_star_percentage    0
3_star_percentage    0
2_star_percentage    0
1_star_percentage    0
dtype: int64


In [ ]:
# df_reviews.to_csv("review_classes.csv",index=False)